**Warning!**

*To activate the C# kernel, follow these steps:*

* Run the code of the next cell and wait for its completion.

* Refresh the browser page (in most cases, this can be done by pressing the F5 key).

* **Run your C# cells!**

In [ ]:
#@title Prepare Google Colab for C# Kernel

# Install dotnet

!wget -q https://packages.microsoft.com/config/ubuntu/20.04/packages-microsoft-prod.deb -O packages-microsoft-prod.deb
!sudo dpkg -i packages-microsoft-prod.deb &> /dev/null
!rm packages-microsoft-prod.deb
!sudo apt-get update -qq &> /dev/null
!sudo apt-get install dotnet-sdk-7.0 -qq &> /dev/null

# Install jupyter kernel

!dotnet tool install -v q --tool-path /usr/bin --version 1.0.360602 Microsoft.dotnet-interactive &> /dev/null
!dotnet interactive jupyter install &> /dev/null

# Create proxy file

with open("ipc_proxy_kernel.py", "w") as f:
    f.write("""import zmq;
import json;
import argparse;
from threading import Thread;
from traitlets.traitlets import Type;
from jupyter_client import KernelClient;
from jupyter_client.session import Session;
from jupyter_client.channels import HBChannel;
from jupyter_client.manager import KernelManager;

parser = argparse.ArgumentParser();
parser.add_argument("connection_file");
parser.add_argument("--kernel", type = str, required = True);
args = parser.parse_args();

# parse connection file details
with open(args.connection_file, "r") as connection_file:
    connection_file_contents = json.load(connection_file);
    transport = str(connection_file_contents["transport"]);
    ip = str(connection_file_contents["ip"]);
    shell_port = int(connection_file_contents["shell_port"]);
    stdin_port = int(connection_file_contents["stdin_port"]);
    control_port = int(connection_file_contents["control_port"]);
    iopub_port = int(connection_file_contents["iopub_port"]);
    hb_port = int(connection_file_contents["hb_port"]);
    signature_scheme = str(connection_file_contents["signature_scheme"]);
    key = str(connection_file_contents["key"]).encode();
# channel | kernel_type | client_type
# shell   | ROUTER      | DEALER
# stdin   | ROUTER      | DEALER
# ctrl    | ROUTER      | DEALER
# iopub   | PUB         | SUB
# hb      | REP         | REQ
zmq_context = zmq.Context()

def create_and_bind_socket(port: int, socket_type: int):
    if(port <= 0):
        raise ValueError(f"Invalid port: {port}");
    if(transport == "tcp"):
        addr = f"tcp://{ip}:{port}";
    elif(transport == "ipc"):
        addr = f"ipc://{ip}-{port}";
    else:
        raise ValueError(f"Unknown transport: {transport}");
    socket: zmq.Socket = zmq_context.socket(socket_type);
    socket.linger = 1000; # ipykernel does this
    socket.bind(addr);
    return socket;

shell_socket = create_and_bind_socket(shell_port, zmq.ROUTER);
stdin_socket = create_and_bind_socket(stdin_port, zmq.ROUTER);
control_socket = create_and_bind_socket(control_port, zmq.ROUTER);
iopub_socket = create_and_bind_socket(iopub_port, zmq.PUB);
hb_socket = create_and_bind_socket(hb_port, zmq.REP);
# Proxy and the real kernel have their own heartbeats. (shoutout to ipykernel
# for this neat little heartbeat implementation)
Thread(target = zmq.device, args = (zmq.QUEUE, hb_socket, hb_socket)).start();

def ZMQProxyChannel_factory(proxy_server_socket: zmq.Socket):
    class ZMQProxyChannel(object):
        kernel_client_socket: zmq.Socket = None;
        session: Session = None;

        def __init__(self, socket: zmq.Socket, session: Session, _ = None):
            super().__init__();
            self.kernel_client_socket = socket;
            self.session = session;

        def start(self):
            # Very convenient zmq device here, proxy will handle the actual zmq
            # proxying on each of our connected sockets (other than heartbeat).
            # It blocks while they are connected so stick it in a thread.
            Thread(
                target = zmq.proxy,
                args = (proxy_server_socket, self.kernel_client_socket)
            ).start();

        def stop(self):
            if(self.kernel_client_socket is not None):
                try:
                    self.kernel_client_socket.close(linger = 0);
                except Exception:
                    pass;
                self.kernel_client_socket = None;

        def is_alive(self):
            return self.kernel_client_socket is not None;

    return ZMQProxyChannel

class ProxyKernelClient(KernelClient):
    shell_channel_class = Type(ZMQProxyChannel_factory(shell_socket));
    stdin_channel_class = Type(ZMQProxyChannel_factory(stdin_socket));
    control_channel_class = Type(ZMQProxyChannel_factory(control_socket));
    iopub_channel_class = Type(ZMQProxyChannel_factory(iopub_socket));
    hb_channel_class = Type(HBChannel);


kernel_manager = KernelManager();
kernel_manager.kernel_name = args.kernel;
kernel_manager.transport = "tcp";
kernel_manager.client_factory = ProxyKernelClient;
kernel_manager.autorestart = False;
# Make sure the wrapped kernel uses the same session info. This way we don't
# need to decode them before forwarding, we can directly pass everything
# through.
kernel_manager.session.signature_scheme = signature_scheme;
kernel_manager.session.key = key;
kernel_manager.start_kernel();
# Connect to the real kernel we just started and start up all the proxies.
kernel_client: ProxyKernelClient = kernel_manager.client();
kernel_client.start_channels();
# Everything should be up and running. We now just wait for the managed kernel
# process to exit and when that happens, shutdown and exit with the same code.
exit_code = kernel_manager.kernel.wait();
kernel_client.stop_channels();
zmq_context.destroy(0);
exit(exit_code);""");

# Install proxy

import os;
import sys;
import json;
import shutil;
import os.path;
from jupyter_client.kernelspec import KernelSpec, KernelSpecManager, NoSuchKernel;

kernel_spec_manager = KernelSpecManager();
real_kernel_spec: KernelSpec = kernel_spec_manager.get_kernel_spec(".net-csharp");
real_kernel_install_path = real_kernel_spec.resource_dir;
new_kernel_name = ".net-csharp_tcp";
new_kernel_install_path = os.path.join(
    os.path.dirname(real_kernel_install_path), new_kernel_name
);
shutil.move(real_kernel_install_path, new_kernel_install_path);
# Update the moved kernel name and args. We tag it _tcp because the proxy will
# impersonate it and should be the one using the real name.
new_kernel_json_path = os.path.join(new_kernel_install_path, "kernel.json");
with open(new_kernel_json_path, "r") as in_:
    real_kernel_json = json.load(in_);
    real_kernel_json["name"] = new_kernel_name;
    real_kernel_json["argv"] = list(
        map(
            lambda arg: arg.replace(real_kernel_install_path, new_kernel_install_path),
            real_kernel_json["argv"]
        )
    );
with open(new_kernel_json_path, "w") as out:
    json.dump(real_kernel_json, out);
os.makedirs(real_kernel_install_path)
proxy_kernel_implementation_path = os.path.join(
    real_kernel_install_path, "ipc_proxy_kernel.py"
);
proxy_kernel_spec = KernelSpec();
proxy_kernel_spec.argv = [
    sys.executable,
    proxy_kernel_implementation_path,
    "{connection_file}",
    f"--kernel={new_kernel_name}"
];
proxy_kernel_spec.display_name = real_kernel_spec.display_name;
proxy_kernel_spec.interrupt_mode = real_kernel_spec.interrupt_mode or "message";
proxy_kernel_spec.language = real_kernel_spec.language;
proxy_kernel_json_path = os.path.join(real_kernel_install_path, "kernel.json");
with open(proxy_kernel_json_path, "w") as out:
    json.dump(proxy_kernel_spec.to_dict(), out, indent = 2);
new_name = shutil.copy("ipc_proxy_kernel.py", proxy_kernel_implementation_path);

In [ ]:
#@title Refresh the browser page (in most cases, this can be done by pressing the F5 key)


In [ ]:
using System;

In [7]:
unsafe float[,] rand(int m, int n)
{
  var mat = new float[m, n];
  var rand = new Random();
  fixed(float* pmat = mat)
  {
    var elements = m * n;
    for(int i = 0; i < elements; ++i)
    {
      pmat[i] = (float)rand.NextDouble();
    }
  }
  return mat;
}

In [10]:
unsafe float[,] matmul(float[,] a, float[,] b)
{
  var m = a.GetLength(0);
  var n = b.GetLength(1);
  var k = b.GetLength(0);
  var c = new float[m, n];
  fixed(float* pa = a)
  {
    fixed(float* pb = b)
    {
      fixed(float* pc = c)
      {
        for(int i = 0; i < m; ++i)
        {
          for(int l = 0; l < k; ++l)
          {
            var pb_ = pb + l * n;
            var pc_ = pc + i * n;
            var av = pa[i * k + l];
            for(int j = 0; j < n; ++j)
            {
              pc_[j] += av * pb_[j];
            }
          }
        }
      }
    }
  }
  return c;
}

In [11]:
var a = rand(1024, 1024);
var b = rand(1024, 1024);
var ms = DateTime.Now;
var ab = matmul(a, b);
Console.WriteLine((DateTime.Now - ms).TotalMilliseconds);

4289.9855
